# Librerias empleadas

In [1]:
# Módulos importantes
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Obteniendo datos

In [2]:
df_train_test = pd.read_csv('german_train_test.txt', delim_whitespace = True, header = None) #, names = columnas)
df_train_test.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,1,6,4,12,5,5,3,4,1,67,...,0,0,1,0,0,1,0,0,1,1
1,2,48,2,60,1,3,2,2,1,22,...,0,0,1,0,0,1,0,0,1,2
2,4,12,4,21,1,4,3,3,1,49,...,0,0,1,0,0,1,0,1,0,1
3,1,42,2,79,1,4,3,4,2,45,...,0,0,0,0,0,0,0,0,1,1
4,1,24,3,49,1,3,3,4,4,53,...,1,0,1,0,0,0,0,0,1,2


In [3]:
df_train_test.isnull().any().any()

False

In [4]:
# Creando X
X= df_train_test.drop(24, axis = 1)

# Creando y
y = df_train_test[24]

In [5]:
df_validation = pd.read_csv('german_validation.txt', delim_whitespace = True, header = None) #, names = columnas)
df_validation.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,1,16,4,26,1,5,3,4,2,43,...,1,0,0,0,1,0,0,0,1,2
1,4,20,4,35,5,2,1,4,1,44,...,1,0,1,0,0,1,0,0,1,1
2,4,36,4,105,5,5,3,4,4,42,...,0,1,1,0,0,0,0,0,1,1
3,4,15,2,14,5,3,4,2,1,40,...,0,0,1,0,1,0,0,0,1,1
4,4,24,2,13,1,5,3,1,1,36,...,0,0,1,0,0,1,0,0,0,1


In [6]:
df_validation.isnull().any().any()

False

In [7]:
# Creando X_validation
X_validation= df_validation.drop(24, axis = 1)

# Creando y_validation
y_validation = df_validation[24]

In [8]:
# Dividiendo datos en datos de entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8)

In [9]:
# Revisando  el tamaño de cada muestra
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((720, 24), (180, 24), (720,), (180,))

# Pipeline

In [10]:
# Modulos importantes
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

In [11]:
# Mostrando columnas
df_train_test.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24],
           dtype='int64')

In [12]:
# Crear la lista de  viriables numericas  numeric_list 
numeric_list = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

In [13]:
# Crear el pipeline de preprocesamiento para datos numéricos  
numeric_transformer = Pipeline(steps = [('scaler', StandardScaler())])

In [14]:
# Definir los transformadores
transformer = ColumnTransformer([('num', numeric_transformer, numeric_list)])

# Modelos

## Árbol de decisión

In [15]:
# Módulos del Modelo
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# modelo DecisionTreeClassifier llamado DT_Classifier
DT_Classifier  = DecisionTreeClassifier()

# Definiendo el modelo_DT con su pipeline
model_DT = Pipeline(  [
    ('transformer',transformer),
    ('DT_Classifier',DT_Classifier)
])

In [16]:
# Ajuste del modelo
model_DT.fit(X_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12, 13, 14, 15, 16,
                                                   17, 18, 19, 20, 21, 22,
                                                   23])])),
                ('DT_Classifier', DecisionTreeClassifier())])

In [17]:
# Score
model_DT.score(X_test, y_test)

0.6444444444444445

In [18]:
from sklearn.model_selection import GridSearchCV

In [19]:
# Definir los parametros con el diccionario params utilizando:
max_depth = [8, 10, 15, 20, 25, 30, 35]
min_samples_leaf = [10, 12, 14, 16, 18, 20]
params = {'DT_Classifier__max_depth':max_depth,
          'DT_Classifier__min_samples_leaf':min_samples_leaf
}

In [20]:
# Ajuste de los parametros max_depth y min_samples_leaf con  GridSearchCV
grid_DT = GridSearchCV(model_DT, param_grid = params, 
                       cv = 10,
                       scoring='neg_mean_squared_error',
                       n_jobs=-1,
                       verbose=2)

In [21]:
# Ajuste del modelo usando todas las posibles combinaciones de parametros 
grid_DT.fit(X, y)

Fitting 10 folds for each of 42 candidates, totalling 420 fits


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         [0, 1,
                                                                          2, 3,
                                                                          4, 5,
                                                                          6, 7,
                                                                          8, 9,
                                                                          10,
                                                                          11,
                                                                          12,
                                                                          13,
                                                                          14,
                                                                          15,
                                                                          16,
                                                                          17,
                                                                          18,
                                                                          19,
                                                                          20,
                                                                          21,
                                                                          22,
                                                                          23])])),
                                       ('DT_Classifier',
                                        DecisionTreeClassifier())]),
             n_jobs=-1,
             param_grid={'DT_Classifier__max_depth': [8, 10, 15, 20, 25, 30,
                                                      35],
                         'DT_Classifier__min_samples_leaf': [10, 12, 14, 16, 18,
                                                             20]},
             scoring='neg_mean_squared_error', verbose=2)

In [22]:
#Encontrar los mejores parametros con best_params_
grid_DT.best_params_

{'DT_Classifier__max_depth': 20, 'DT_Classifier__min_samples_leaf': 14}

In [23]:
# Ajustar el modelo con los nuevos parametros nombrandolo como  DT_Classifier_tunned
DT_Classifier_tunned  = DecisionTreeClassifier(max_depth = grid_DT.best_params_['DT_Classifier__max_depth'], 
                                          min_samples_leaf = grid_DT.best_params_['DT_Classifier__min_samples_leaf'])

In [24]:
# Definiendo el modelo_DT con su pipeline
model_DT_tunned = Pipeline(  [
    ('transformer',transformer),
    ('DT_Classifier',DT_Classifier_tunned)
])

In [25]:
#3. Ajustar el modelo
model_DT_tunned.fit(X_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12, 13, 14, 15, 16,
                                                   17, 18, 19, 20, 21, 22,
                                                   23])])),
                ('DT_Classifier',
                 DecisionTreeClassifier(max_depth=20, min_samples_leaf=14))])

In [26]:
#4 Score
model_DT_tunned.score(X_test, y_test)

0.6888888888888889

## RandomForest

In [27]:
# Módulos del modelo
from sklearn.ensemble import RandomForestClassifier

In [28]:
# Asignamos nuestro modelo a una variable
forest_clf = RandomForestClassifier(random_state=42)

In [29]:
# Definiendo el Pipeline
model_forest_clf = Pipeline(steps=[
    ('transformer', transformer),
    ('forest_clf', forest_clf)
])

In [30]:
# Ajustar el modelo
model_forest_clf.fit(X_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12, 13, 14, 15, 16,
                                                   17, 18, 19, 20, 21, 22,
                                                   23])])),
                ('forest_clf', RandomForestClassifier(random_state=42))])

In [31]:
model_forest_clf.score(X_test, y_test)

0.7277777777777777

In [32]:
from sklearn.model_selection import GridSearchCV

In [33]:
#Definición de posibles parametros
# n_estimators = [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]
# max_features = ['auto', 'sqrt']
# max_depth = [int(x) for x in np.linspace(3, 30, num = 3)]
# min_samples_split = [2,3,4,5,6 ]
# min_samples_leaf = [1, 2, 3,4,5,6 ]

max_depth = [8, 10, 15, 20, 25, 30, 35]
min_samples_leaf = [10, 12, 14, 16, 18, 20]
n_estimators = [3, 30, 80, 100, 150, 200]

params = {'forest_clf__n_estimators': n_estimators,
          'forest_clf__max_depth': max_depth,
          'forest_clf__min_samples_leaf': min_samples_leaf
         }

# params = {'forest_clf__n_estimators': n_estimators,
#           'forest_clf__max_features': max_features,
#           'forest_clf__max_depth': max_depth,
#           'forest_clf__min_samples_split': min_samples_split,
#           'forest_clf__min_samples_leaf': min_samples_leaf
#          }

In [34]:
#Definicion del grid de busqueda
grid_RF = GridSearchCV(model_forest_clf, param_grid=params,
                   cv=10, scoring='neg_mean_squared_error',
                   n_jobs=-1, verbose=2)

In [35]:
# Ajuste del modelo usando todas las posibles combinaciones de parametros 
grid_RF.fit(X, y)

Fitting 10 folds for each of 252 candidates, totalling 2520 fits


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         [0, 1,
                                                                          2, 3,
                                                                          4, 5,
                                                                          6, 7,
                                                                          8, 9,
                                                                          10,
                                                                          11,
                                                                          12,
                                                                          13,
                                                                          14,
                                                                          15,
                                                                          16,
                                                                          17,
                                                                          18,
                                                                          19,
                                                                          20,
                                                                          21,
                                                                          22,
                                                                          23])])),
                                       ('forest_clf',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'forest_clf__max_depth': [8, 10, 15, 20, 25, 30, 35],
                         'forest_clf__min_samples_leaf': [10, 12, 14, 16, 18,
                                                          20],
                         'forest_clf__n_estimators': [3, 30, 80, 100, 150,
                                                      200]},
             scoring='neg_mean_squared_error', verbose=2)

In [36]:
# Mostrar la mejor combinación
grid_RF.best_params_

{'forest_clf__max_depth': 8,
 'forest_clf__min_samples_leaf': 10,
 'forest_clf__n_estimators': 30}

In [37]:
# Ajustar el modelo con los nuevos parametros nombrandolo como  DT_Classifier_tunned
# model_forest_clf_tunned  = RandomForestClassifier(max_depth = grid_RF.best_params_['forest_clf__max_depth'], 
#                                                  max_features = grid_RF.best_params_['forest_clf__max_features'],
#                                                  min_samples_leaf = grid_RF.best_params_['forest_clf__min_samples_leaf'], 
#                                                  min_samples_split = grid_RF.best_params_['forest_clf__min_samples_split'],
#                                                  n_estimators = grid_RF.best_params_['forest_clf__n_estimators'])

forest_clf_tunned  = RandomForestClassifier(max_depth = grid_RF.best_params_['forest_clf__max_depth'], 
                                                 min_samples_leaf = grid_RF.best_params_['forest_clf__min_samples_leaf'],
                                                 n_estimators = grid_RF.best_params_['forest_clf__n_estimators'])

model_forest_clf_tunned = Pipeline(steps=[
    ('transformer', transformer),
    ('forest_clf', forest_clf_tunned)
])

In [38]:
#3. Ajustar el modelo
model_forest_clf_tunned.fit(X_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12, 13, 14, 15, 16,
                                                   17, 18, 19, 20, 21, 22,
                                                   23])])),
                ('forest_clf',
                 RandomForestClassifier(max_depth=8, min_samples_leaf=10,
                                        n_estimators=30))])

In [39]:
model_forest_clf_tunned.score(X_test, y_test)

0.6833333333333333